In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')




## Dataframe Importieren


In [2]:

holidays_events = pd.read_csv("./database/holidays_events.csv")
train = pd.read_csv('./database/train.csv')
test = pd.read_csv('./database/test.csv')
transactions = pd.read_csv('./database/transactions.csv')
stores = pd.read_csv('./database/stores.csv')
oil = pd.read_csv('./database/oil.csv')
sample_submission = pd.read_csv('./database/sample_submission.csv')

## Dataframe Kombinieren
le dataframe actuel contient seulement les ventes actuelles. mais n'a pas les informations sur les stores , oil , holiday ,transactions. 




In [3]:
train_dataset = train.merge(oil, on = 'date', how= 'left')
train_dataset = train_dataset.merge(holidays_events, on = 'date', how = 'left')
train_dataset = train_dataset.merge(stores, on = 'store_nbr', how = 'left')
train_dataset = train_dataset.merge(transactions, on = ['date','store_nbr'], how= 'left')
train_dataset.head()


,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,city,state,type_y,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN


In [4]:
train_dataset = train_dataset.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})
train_dataset.head()


,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN


In [5]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 17 columns):
id              int64
date            object
store_nbr       int64
family          object
sales           float64
onpromotion     int64
dcoilwtico      float64
holiday_type    object
locale          object
locale_name     object
description     object
transferred     object
city            object
state           object
store_type      object
cluster         int64
transactions    float64
dtypes: float64(3), int64(4), object(10)
memory usage: 419.5+ MB


In [6]:
test_dataframe = test.merge(oil, on = 'date', how='left')
test_dataframe = test_dataframe.merge(holidays_events, on = 'date', how='left')
test_dataframe = test_dataframe.merge(stores, on = 'store_nbr', how='left')
test_dataframe = test_dataframe.merge(transactions, on = ['date', 'store_nbr'], how='left')
test_dataframe = test_dataframe.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})
test_dataframe.head()
test_dataframe.info()




<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 16 columns):
id              28512 non-null int64
date            28512 non-null object
store_nbr       28512 non-null int64
family          28512 non-null object
onpromotion     28512 non-null int64
dcoilwtico      21384 non-null float64
holiday_type    1782 non-null object
locale          1782 non-null object
locale_name     1782 non-null object
description     1782 non-null object
transferred     1782 non-null object
city            28512 non-null object
state           28512 non-null object
store_type      28512 non-null object
cluster         28512 non-null int64
transactions    0 non-null float64
dtypes: float64(2), int64(4), object(10)
memory usage: 3.7+ MB


## Spalte 'Date' in Datetime convertieren 
## Verschiedene Datum anhand Features hinzufügen 

In [7]:
def add_date_features(df):
    
    df['date'] = pd.to_datetime(df['date'])
    
    df['dayofweek'] = df['date'].dt.dayofweek  # Day of the week (Monday is 0 and Sunday is 6)
    df['quarter'] = df['date'].dt.quarter      # Quarter of the year
    df['month'] = df['date'].dt.month          # Month (1 to 12)
    df['year'] = df['date'].dt.year            # Year
    df['dayofyear'] = df['date'].dt.dayofyear  # Day of the year
    df['dayofmonth'] = df['date'].dt.day       # Day of the month

    return df

In [8]:
train_dataset= add_date_features(train_dataset)
test_dataframe= add_date_features(test_dataframe)
train_dataset.tail(5)

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,holiday_type,locale,locale_name,...,state,store_type,cluster,transactions,dayofweek,quarter,month,year,dayofyear,dayofmonth
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Holiday,Local,Riobamba,...,Pichincha,B,6,2155.0,1,3,8,2017,227,15
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Holiday,Local,Riobamba,...,Pichincha,B,6,2155.0,1,3,8,2017,227,15
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Holiday,Local,Riobamba,...,Pichincha,B,6,2155.0,1,3,8,2017,227,15
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Holiday,Local,Riobamba,...,Pichincha,B,6,2155.0,1,3,8,2017,227,15
3054347,3000887,2017-08-15,9,SEAFOOD,16.000,0,47.57,Holiday,Local,Riobamba,...,Pichincha,B,6,2155.0,1,3,8,2017,227,15


In [9]:
test_dataframe.tail(5)

,id,date,store_nbr,family,onpromotion,dcoilwtico,holiday_type,locale,locale_name,description,...,state,store_type,cluster,transactions,dayofweek,quarter,month,year,dayofyear,dayofmonth
28507,3029395,2017-08-31,9,POULTRY,1,47.26,NaN,NaN,NaN,NaN,...,Pichincha,B,6,NaN,3,3,8,2017,243,31
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,NaN,NaN,NaN,NaN,...,Pichincha,B,6,NaN,3,3,8,2017,243,31
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,NaN,NaN,NaN,NaN,...,Pichincha,B,6,NaN,3,3,8,2017,243,31
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,NaN,NaN,NaN,NaN,...,Pichincha,B,6,NaN,3,3,8,2017,243,31
28511,3029399,2017-08-31,9,SEAFOOD,0,47.26,NaN,NaN,NaN,NaN,...,Pichincha,B,6,NaN,3,3,8,2017,243,31


In [10]:
train_dataset.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
dcoilwtico       955152
holiday_type    2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
city                  0
state                 0
store_type            0
cluster               0
transactions     249117
dayofweek             0
quarter               0
month                 0
year                  0
dayofyear             0
dayofmonth            0
dtype: int64

## transfered Tag finden 

In [11]:
tranfered_days = {date: item for date, item in zip(holidays_events['date'], holidays_events['transferred']) if item}
tranfered_days


{'2012-10-09': True,
 '2013-10-09': True,
 '2014-10-09': True,
 '2016-05-24': True,
 '2016-07-25': True,
 '2016-08-10': True,
 '2017-01-01': True,
 '2017-04-12': True,
 '2017-05-24': True,
 '2017-08-10': True,
 '2017-09-28': True,
 '2017-12-06': True}

In [12]:
train_dataset.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,holiday_type,locale,locale_name,...,state,store_type,cluster,transactions,dayofweek,quarter,month,year,dayofyear,dayofmonth
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1


## prüfen, ob es ein Tranferred Datum im Train_dataset gibt

In [13]:

train_dataset['transferred'] = train_dataset['date'].map(tranfered_days)
train_dataset['transferred'] = train_dataset['transferred'].fillna(value=False)
test_dataframe['transferred'] = test_dataframe['date'].map(tranfered_days)
test_dataframe['transferred'] = test_dataframe['transferred'].fillna(value=False)

In [14]:
train_dataset.head()
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 23 columns):
id              int64
date            datetime64[ns]
store_nbr       int64
family          object
sales           float64
onpromotion     int64
dcoilwtico      float64
holiday_type    object
locale          object
locale_name     object
description     object
transferred     bool
city            object
state           object
store_type      object
cluster         int64
transactions    float64
dayofweek       int64
quarter         int64
month           int64
year            int64
dayofyear       int64
dayofmonth      int64
dtypes: bool(1), datetime64[ns](1), float64(3), int64(10), object(8)
memory usage: 538.9+ MB


## null Werte prüfen 

In [15]:
print(train_dataset.isnull().sum())

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
dcoilwtico       955152
holiday_type    2551824
locale          2551824
locale_name     2551824
description     2551824
transferred           0
city                  0
state                 0
store_type            0
cluster               0
transactions     249117
dayofweek             0
quarter               0
month                 0
year                  0
dayofyear             0
dayofmonth            0
dtype: int64


In [16]:
print(test_dataframe.isnull().sum())

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
dcoilwtico       7128
holiday_type    26730
locale          26730
locale_name     26730
description     26730
transferred         0
city                0
state               0
store_type          0
cluster             0
transactions    28512
dayofweek           0
quarter             0
month               0
year                0
dayofyear           0
dayofmonth          0
dtype: int64


In [17]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Daten Encoding 

 

In [20]:
from sklearn.preprocessing import LabelEncoder

In [18]:
train_dataset.info()
train_dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 23 columns):
id              int64
date            datetime64[ns]
store_nbr       int64
family          object
sales           float64
onpromotion     int64
dcoilwtico      float64
holiday_type    object
locale          object
locale_name     object
description     object
transferred     bool
city            object
state           object
store_type      object
cluster         int64
transactions    float64
dayofweek       int64
quarter         int64
month           int64
year            int64
dayofyear       int64
dayofmonth      int64
dtypes: bool(1), datetime64[ns](1), float64(3), int64(10), object(8)
memory usage: 538.9+ MB


,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,holiday_type,locale,locale_name,...,state,store_type,cluster,transactions,dayofweek,quarter,month,year,dayofyear,dayofmonth
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Holiday,National,Ecuador,...,Pichincha,D,13,NaN,1,1,1,2013,1,1
